<a href="https://colab.research.google.com/github/IbHansen/wb-debt-simulation/blob/main/simulation/Specification_debt_simulation_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
#This is code to manage dependencies if the notebook is executed in the google colab cloud service
if 'google.colab' in str(get_ipython()):
  import os
  os.system('apt -qqq install graphviz')
  os.system('pip -qqq install ModelFlowIb')

In [38]:
import modeljupytermagic

In [39]:
import pandas as pd
import re

from modelclass import model
import modelmanipulation as mp
from modelmanipulation import doable, explode

from modelpattern import list_extract

# About lists

**lists** and **sublists** are central to making ModelFlow scalable and expressive.  
They allow the modeler to define **groups of entities** (such as bonds, banks, portfolios, sectors, or countries) and to attach **attributes** or **hierarchies** to those entities.  
A list definition typically takes the following form

$list\: \underbrace{issued}_{List name} = \underbrace{issued}_{sub listname} : issued\_2025 * issued\_2050 / \\
\underbrace{issued\_year}_{sub listname} : 2025 * 2050$

Please note:
 - the list name and the first sub list name should be the same
 - case don't matter
 - a * combined with identifiers ending in digits will construct multiple list content

  `issued_2025 * issued_2050` becomes `issued_2025 issued_2026 … issued_2050`


This enables equations to be written once as templates and then **expanded automatically** across all relevant list members.  

Sublists can be used to define logical relationships between entities — for example, linking each bond to its home maturity, or to define the year for each issurance year.  
These structures make it possible to organize models systematically and to maintain consistency across entities that share the same behavioral equations.  

This approach eliminates redundancy and manual repetition: instead of writing dozens or hundreds of similar equations, the modeler defines a **single equation pattern**, and ModelFlow expands it dynamically over the members of a list.  

In [40]:
%%Mexplodemodel port segment=listport
# List definitions


> list issued = issued        : issued_2025 * issued_2050 /
>>              issued_year   :        2025 *        2050

>list bond = bond :     1_year_dom 5_year_dom 10_year_dom  1_year_usd 5_year_usd 10_year_usd /
>>        maturity :              1          5          10           1          5          10 /
>>        currency :            dom        dom         dom         usd        usd         usd /
>>         domestic :             1          1           1           0          0           0 /
>>         foreign :              0          0           0           1          1           1

# List definitions


```text
> list issued = issued        : issued_2025 * issued_2050 /
>>              issued_year   :        2025 *        2050
```

```text
>list bond = bond :     1_year_dom 5_year_dom 10_year_dom  1_year_usd 5_year_usd 10_year_usd /
>>        maturity :              1          5          10           1          5          10 /
>>        currency :            dom        dom         dom         usd        usd         usd /
>>         domestic :             1          1           1           0          0           0 /
>>         foreign :              0          0           0           1          1           1
```

In [41]:
port_replacements=[('__bond','__{bond}__{issued}'),
                   ('fx_rate','((dom_{currency}>0.000001) + (dom_{currency} < 0.000001))')
                  ]

In [42]:
%%Mexplodemodel port segment=logical replacements=port_replacements

## Define logical variables
The folowing variables are defined for each bond and each issuing year. They are **logical variables** that is they can be either 0.0 or 1.0.

The terms enclosed in `{}` are taken from the relevant sublists. the  `current_year` are taken from the input dataset.

> doable logical_issuing__bond = current_year == {issued_year}
> doable logical_maturing__bond =  current_year == {issued_year}+{maturity}

> doable logical_paying_years__bond =  {issued_year} < current_year <= ({issued_year} + {maturity})



# List definitions


```text
> list issued = issued        : issued_2025 * issued_2050 /
>>              issued_year   :        2025 *        2050
```

```text
>list bond = bond :     1_year_dom 5_year_dom 10_year_dom  1_year_usd 5_year_usd 10_year_usd /
>>        maturity :              1          5          10           1          5          10 /
>>        currency :            dom        dom         dom         usd        usd         usd /
>>         domestic :             1          1           1           0          0           0 /
>>         foreign :              0          0           0           1          1           1
```

## Define logical variables
The folowing variables are defined for each bond and each issuing year. They are **logical variables** that is they can be either 0.0 or 1.0.

The terms enclosed in `{}` are taken from the relevant sublists. the  `current_year` are taken from the input dataset.

```text
> doable logical_issuing__bond = current_year == {issued_year}
> doable logical_maturing__bond =  current_year == {issued_year}+{maturity}
```

```text
> doable logical_paying_years__bond =  {issued_year} < current_year <= ({issued_year} + {maturity})
```


In [43]:
%%Mexplodemodel port segment=core replacements=port_replacements

## Funding need
>funding_need = deficit+amortizing__all + interest_payments__all

## Interest rates
The first year after the issurance the interest rate is taken from the interest rate of the relevant maturity.

In the rest of the paying years the interest rate are set to the bonds interest rate the previous year
> doable [bond=DOMESTIC] interest_rate__bond = interest_rate__{maturity}(-1)            * logical_issuing__bond (-1) + interest_rate__bond(-1) * logical_paying_years__bond
> doable [bond=foreign]  interest_rate__bond = interest_rate_{currency}__{maturity}(-1) * logical_issuing__bond (-1) + interest_rate__bond(-1) * logical_paying_years__bond


## in_currency

### issue in the individual bonds
> doable new_issue_in_currency__bond = new_issue_share__{bond}/100 * funding_need * logical_issuing__bond  /  fx_rate

### Amortizing
> doable  amortizing_in_currency__bond = stock_primo_in_currency__bond * logical_maturing__bond


### End of year stock
> doable stock_ultimo_in_currency__bond = stock_primo_in_currency__bond+ new_issue_in_currency__bond-amortizing_in_currency__bond

### Start of year stock
> doable stock_primo_in_currency__bond = stock_ultimo_in_currency__bond(-1)

### Interest rate payment
> doable interest_in_currency__bond = stock_primo_in_currency__bond * interest_rate__bond/100

## Domestic
### Interest rate payment
> doable <sum=all>  interest_payments__bond    = interest_in_currency__bond  * fx_rate

### Amortizing

> doable <sum=all>  amortizing__bond = amortizing_in_currency__bond * fx_rate

### Stock
> doable <sum=all> stock_primo__bond = stock_primo_in_currency__bond * fx_rate
> doable <sum=all> stock_ultimo__bond = stock_ultimo_in_currency__bond * fx_rate



# List definitions


```text
> list issued = issued        : issued_2025 * issued_2050 /
>>              issued_year   :        2025 *        2050
```

```text
>list bond = bond :     1_year_dom 5_year_dom 10_year_dom  1_year_usd 5_year_usd 10_year_usd /
>>        maturity :              1          5          10           1          5          10 /
>>        currency :            dom        dom         dom         usd        usd         usd /
>>         domestic :             1          1           1           0          0           0 /
>>         foreign :              0          0           0           1          1           1
```

## Funding need
```text
>funding_need = deficit+amortizing__all + interest_payments__all
```

## Interest rates
The first year after the issurance the interest rate is taken from the interest rate of the relevant maturity.

In the rest of the paying years the interest rate are set to the bonds interest rate the previous year
```text
> doable [bond=DOMESTIC] interest_rate__bond = interest_rate__{maturity}(-1)            * logical_issuing__bond (-1) + interest_rate__bond(-1) * logical_paying_years__bond
> doable [bond=foreign]  interest_rate__bond = interest_rate_{currency}__{maturity}(-1) * logical_issuing__bond (-1) + interest_rate__bond(-1) * logical_paying_years__bond
```


## in_currency

### issue in the individual bonds
```text
> doable new_issue_in_currency__bond = new_issue_share__{bond}/100 * funding_need * logical_issuing__bond  /  fx_rate
```

### Amortizing
```text
> doable  amortizing_in_currency__bond = stock_primo_in_currency__bond * logical_maturing__bond
```


### End of year stock
```text
> doable stock_ultimo_in_currency__bond = stock_primo_in_currency__bond+ new_issue_in_currency__bond-amortizing_in_currency__bond
```

### Start of year stock
```text
> doable stock_primo_in_currency__bond = stock_ultimo_in_currency__bond(-1)
```

### Interest rate payment
```text
> doable interest_in_currency__bond = stock_primo_in_currency__bond * interest_rate__bond/100
```

## Domestic
### Interest rate payment
```text
> doable <sum=all>  interest_payments__bond    = interest_in_currency__bond  * fx_rate
```

### Amortizing

```text
> doable <sum=all>  amortizing__bond = amortizing_in_currency__bond * fx_rate
```

### Stock
```text
> doable <sum=all> stock_primo__bond = stock_primo_in_currency__bond * fx_rate
> doable <sum=all> stock_ultimo__bond = stock_ultimo_in_currency__bond * fx_rate
```


In [44]:
port_dict.keys()

dict_keys(['listport', 'logical', 'core'])

In [45]:
%Mexplodemodel port replacements=port_replacements
mport = port.mmodel


## Define logical variables
The folowing variables are defined for each bond and each issuing year. They are **logical variables** that is they can be either 0.0 or 1.0.

The terms enclosed in `{}` are taken from the relevant sublists. the  `current_year` are taken from the input dataset.

```text
> doable logical_issuing__bond = current_year == {issued_year}
> doable logical_maturing__bond =  current_year == {issued_year}+{maturity}
```

```text
> doable logical_paying_years__bond =  {issued_year} < current_year <= ({issued_year} + {maturity})
```



## Funding need
```text
>funding_need = deficit+amortizing__all + interest_payments__all
```

## Interest rates
The first year after the issurance the interest rate is taken from the interest rate of the relevant maturity.

In the rest of the paying years the interest rate are set to the bonds interest rate the previous year
```text
> doable [bond=DOMESTIC] interest_rate__bond = interest_rate__{maturity}(-1)            * logical_issuing__bond (-1) + interest_rate__bond(-1) * logical_paying_years__bond
> doable [bond=foreign]  interest_rate__bond = interest_rate_{currency}__{maturity}(-1) * logical_issuing__bond (-1) + interest_rate__bond(-1) * logical_paying_years__bond
```


## in_currency

### issue in the individual bonds
```text
> doable new_issue_in_currency__bond = new_issue_share__{bond}/100 * funding_need * logical_issuing__bond  /  fx_rate
```

### Amortizing
```text
> doable  amortizing_in_currency__bond = stock_primo_in_currency__bond * logical_maturing__bond
```


### End of year stock
```text
> doable stock_ultimo_in_currency__bond = stock_primo_in_currency__bond+ new_issue_in_currency__bond-amortizing_in_currency__bond
```

### Start of year stock
```text
> doable stock_primo_in_currency__bond = stock_ultimo_in_currency__bond(-1)
```

### Interest rate payment
```text
> doable interest_in_currency__bond = stock_primo_in_currency__bond * interest_rate__bond/100
```

## Domestic
### Interest rate payment
```text
> doable <sum=all>  interest_payments__bond    = interest_in_currency__bond  * fx_rate
```

### Amortizing

```text
> doable <sum=all>  amortizing__bond = amortizing_in_currency__bond * fx_rate
```

### Stock
```text
> doable <sum=all> stock_primo__bond = stock_primo_in_currency__bond * fx_rate
> doable <sum=all> stock_ultimo__bond = stock_ultimo_in_currency__bond * fx_rate
```


# List definitions


```text
> list issued = issued        : issued_2025 * issued_2050 /
>>              issued_year   :        2025 *        2050
```

```text
>list bond = bond :     1_year_dom 5_year_dom 10_year_dom  1_year_usd 5_year_usd 10_year_usd /
>>        maturity :              1          5          10           1          5          10 /
>>        currency :            dom        dom         dom         usd        usd         usd /
>>         domestic :             1          1           1           0          0           0 /
>>         foreign :              0          0           0           1          1           1
```

In [46]:
years = [y for y in range(2025,2050,1)]
dfstart = pd.DataFrame(years,index=years,columns=['CURRENT_YEAR'])
dfstart

,CURRENT_YEAR
2025,2025
2026,2026
2027,2027
2028,2028
2029,2029
2030,2030
2031,2031
2032,2032
2033,2033
2034,2034


In [47]:
df = dfstart.upd('''
new_issue_share__1_year_dom = 40
new_issue_share__5_year_dom  = 40
new_issue_share__10_year_dom  = 20
interest_rate__1 = 1
interest_rate__5 = 1
interest_rate__10 = 1
interest_rate_usd__1 = 3
interest_rate_usd__5 = 3
interest_rate_usd__10 = 3
<2026> deficit  = 100

''')
df.head()


,CURRENT_YEAR,NEW_ISSUE_SHARE__1_YEAR_DOM,NEW_ISSUE_SHARE__5_YEAR_DOM,NEW_ISSUE_SHARE__10_YEAR_DOM,INTEREST_RATE__1,INTEREST_RATE__5,INTEREST_RATE__10,INTEREST_RATE_USD__1,INTEREST_RATE_USD__5,INTEREST_RATE_USD__10,DEFICIT
2025,2025,40.0,40.0,20.0,1.0,1.0,1.0,3.0,3.0,3.0,0.0
2026,2026,40.0,40.0,20.0,1.0,1.0,1.0,3.0,3.0,3.0,100.0
2027,2027,40.0,40.0,20.0,1.0,1.0,1.0,3.0,3.0,3.0,0.0
2028,2028,40.0,40.0,20.0,1.0,1.0,1.0,3.0,3.0,3.0,0.0
2029,2029,40.0,40.0,20.0,1.0,1.0,1.0,3.0,3.0,3.0,0.0


In [48]:
df.loc[:,mport.vlist('interest_rate__*')] = 1

In [49]:
res = mport(df,silent=1)

In [50]:
mport['INTEREST_RATE__*__*26  interest*__*26 funding* paying_years__*26'].df.T


,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,...,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049
INTEREST_RATE__10_YEAR_DOM__ISSUED_2026,0.0,1.0,1.00,1.0000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,...,0.0000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000
INTEREST_RATE__10_YEAR_USD__ISSUED_2026,0.0,3.0,3.00,3.0000,3.000000,3.00000,3.000000,3.000000,3.000000,3.000000,...,0.0000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000
INTEREST_RATE__1_YEAR_DOM__ISSUED_2026,0.0,1.0,0.00,0.0000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000
INTEREST_RATE__1_YEAR_USD__ISSUED_2026,0.0,3.0,0.00,0.0000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000
INTEREST_RATE__5_YEAR_DOM__ISSUED_2026,0.0,1.0,1.00,1.0000,1.000000,1.00000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000
INTEREST_RATE__5_YEAR_USD__ISSUED_2026,0.0,3.0,3.00,3.0000,3.000000,3.00000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000
INTEREST_IN_CURRENCY__10_YEAR_DOM__ISSUED_2026,0.0,0.2,0.20,0.2000,0.200000,0.20000,0.200000,0.200000,0.200000,0.200000,...,0.0000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000
INTEREST_IN_CURRENCY__10_YEAR_USD__ISSUED_2026,0.0,0.0,0.00,0.0000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000
INTEREST_IN_CURRENCY__1_YEAR_DOM__ISSUED_2026,0.0,0.4,0.00,0.0000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000
INTEREST_IN_CURRENCY__1_YEAR_USD__ISSUED_2026,0.0,0.0,0.00,0.0000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000


In [51]:
mport['stock_primo__all stock_ultimo__all interest*__*26 funding* paying_years__*26'].df.T


,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,...,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049
STOCK_PRIMO__ALL,0.0,100.0,101.00,102.0100,103.030100,104.060401,105.101005,106.152015,107.213535,108.285671,...,113.809328,114.947421,116.096896,117.257864,118.430443,119.614748,120.810895,122.019004,123.239194,124.471586
STOCK_ULTIMO__ALL,100.0,101.0,102.01,103.0301,104.060401,105.101005,106.152015,107.213535,108.285671,109.368527,...,114.947421,116.096896,117.257864,118.430443,119.614748,120.810895,122.019004,123.239194,124.471586,125.716302
INTEREST_IN_CURRENCY__10_YEAR_DOM__ISSUED_2026,0.0,0.2,0.20,0.2000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
INTEREST_IN_CURRENCY__10_YEAR_USD__ISSUED_2026,0.0,0.0,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
INTEREST_IN_CURRENCY__1_YEAR_DOM__ISSUED_2026,0.0,0.4,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
INTEREST_IN_CURRENCY__1_YEAR_USD__ISSUED_2026,0.0,0.0,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
INTEREST_IN_CURRENCY__5_YEAR_DOM__ISSUED_2026,0.0,0.4,0.40,0.4000,0.400000,0.400000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
INTEREST_IN_CURRENCY__5_YEAR_USD__ISSUED_2026,0.0,0.0,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
INTEREST_PAYMENTS__10_YEAR_DOM__ISSUED_2026,0.0,0.2,0.20,0.2000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
INTEREST_PAYMENTS__10_YEAR_USD__ISSUED_2026,0.0,0.0,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [52]:
mport['interest_payments__all interest_in_currency__*202? interest_rate__*202?'].df.head().T

,2026,2027,2028,2029,2030
INTEREST_PAYMENTS__ALL,0.0,1.0,1.010,1.02010,1.030301
INTEREST_IN_CURRENCY__10_YEAR_DOM__ISSUED_2025,0.0,0.0,0.000,0.00000,0.000000
INTEREST_IN_CURRENCY__10_YEAR_DOM__ISSUED_2026,0.0,0.2,0.200,0.20000,0.200000
INTEREST_IN_CURRENCY__10_YEAR_DOM__ISSUED_2027,0.0,0.0,0.082,0.08200,0.082000
INTEREST_IN_CURRENCY__10_YEAR_DOM__ISSUED_2028,0.0,0.0,0.000,0.03482,0.034820
...,...,...,...,...,...
INTEREST_RATE__5_YEAR_USD__ISSUED_2025,1.0,1.0,1.000,1.00000,1.000000
INTEREST_RATE__5_YEAR_USD__ISSUED_2026,0.0,3.0,3.000,3.00000,3.000000
INTEREST_RATE__5_YEAR_USD__ISSUED_2027,0.0,0.0,3.000,3.00000,3.000000
INTEREST_RATE__5_YEAR_USD__ISSUED_2028,0.0,0.0,0.000,3.00000,3.000000


In [53]:
mport.NEW_ISSUE_IN_CURRENCY__1_YEAR_DOM__ISSUED_2026

Endogeneous: NEW_ISSUE_IN_CURRENCY__1_YEAR_DOM__ISSUED_2026: 
Formular: FRML <> NEW_ISSUE_IN_CURRENCY__1_YEAR_DOM__ISSUED_2026 = NEW_ISSUE_SHARE__1_YEAR_DOM/100*FUNDING_NEED*LOGICAL_ISSUING__1_YEAR_DOM__ISSUED_2026/((DOM_DOM>0.000001)+(DOM_DOM<0.000001)) $

NEW_ISSUE_IN_CURRENCY__1_YEAR_DOM__ISSUED_2026: 
DOM_DOM                                       : 
FUNDING_NEED                                  : 
LOGICAL_ISSUING__1_YEAR_DOM__ISSUED_2026      : 
NEW_ISSUE_SHARE__1_YEAR_DOM                   : 

Values :


,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049
Base,40.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Last,40.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Diff,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


Input last run:


,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049
DOM_DOM,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
FUNDING_NEED,100.00,41.00,17.41,7.98,4.22,42.73,34.54,21.84,13.00,7.97,41.38,39.67,29.20,19.61,13.01,31.45,36.52,31.83,24.36,17.74,29.16,35.43,33.98,28.50
LOGICAL_ISSUING__1_YEAR_DOM__ISSUED_2026,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
NEW_ISSUE_SHARE__1_YEAR_DOM,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00


In [54]:
mport.modeldump('model/port')

In [55]:
mport['logical_pay*__5*'].df.T

,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,...,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049
LOGICAL_PAYING_YEARS__5_YEAR_DOM__ISSUED_2025,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LOGICAL_PAYING_YEARS__5_YEAR_DOM__ISSUED_2026,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LOGICAL_PAYING_YEARS__5_YEAR_DOM__ISSUED_2027,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LOGICAL_PAYING_YEARS__5_YEAR_DOM__ISSUED_2028,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LOGICAL_PAYING_YEARS__5_YEAR_DOM__ISSUED_2029,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LOGICAL_PAYING_YEARS__5_YEAR_DOM__ISSUED_2030,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LOGICAL_PAYING_YEARS__5_YEAR_DOM__ISSUED_2031,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LOGICAL_PAYING_YEARS__5_YEAR_DOM__ISSUED_2032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LOGICAL_PAYING_YEARS__5_YEAR_DOM__ISSUED_2033,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LOGICAL_PAYING_YEARS__5_YEAR_DOM__ISSUED_2034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
